''' This program runs a dynamic topic model on future only paragraph

#Date: July 7, 2020

#Author: Carly Knight

#https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/dtm_example.ipynb
#https://stackoverflow.com/questions/50413059/dynamic-topic-modeling-with-gensim-which-code'''


## set up

In [1]:
import logging
import os
from gensim import corpora, utils
from gensim.models.wrappers.dtmmodel import DtmModel
import numpy as np
import pandas as pd
from gensim.corpora import Dictionary
from collections import defaultdict



### dtm

In [2]:
# you can also copy the path down directly. Change this variable to your DTM executable before running.
dtm_path = "/usr/local/bin/dtm-darwin64"

# Open corpus

In [3]:
file_loc = "/Users/carlyknight/Documents/Data/Annual Report/report_paragraphs/future_texts_paragraphs_futureonly/"

In [7]:
#read data
metadata = pd.read_csv(file_loc + "metadata_futureperfect_paragraphs_futureonly.csv") 

In [8]:
#limit to over 1930
metadata = metadata[(metadata['Year'] >= 1930)]

#create vars for filenames
metadata['document_filename'] = metadata['Filename'].str.replace(".xml", ".txt")
metadata['paragraph_filetime']= metadata['Filename'].str.replace(".xml", "") + "_P" + metadata['Paragraph'].astype(str) + ".txt"

#941727, 19
metadata.shape 

(153955, 18)

In [9]:
#pick relevant files
textfiles = [i for i in metadata['paragraph_filetime'].values.tolist()]

In [11]:
len(textfiles)

153955

In [12]:
# #open files (this step takes forever)
# for index, row in metadata.iterrows():
#     txtfile= row['paragraph_filetime']
#     #read in text file and turn into new variable
#     with open(file_loc + txtfile, 'r') as f:
#         metadata.loc[index,'text']= f.read()
#         f.close()

In [13]:
#empty dictionary
documentslist = []

#put txt files into dictionary
for txtfile in textfiles:
    with open(file_loc + txtfile, 'r') as f:
        documentslist.append(f.read())
        f.close()

In [14]:
#remove puncutation
#metadata['text'] = metadata['text'].str.replace(r'[^\w\s]+', '')

#remove numbers
#metadata['text'] = metadata['text'].str.replace(r'[\d]+', '')

#remove extraspaces
#metadata['text'] = metadata['text'].str.replace(r'\s+', ' ')

#turn to text
#documents = metadata['text'].tolist()

#turn dict to list
#documentslist = list(documentsdict.values())

## Preprocess

In [15]:
from gensim.parsing.preprocessing import preprocess_string

#processed corpus: removes punctuation, whitespaces, stopwords, stems, numbes
processed_corpus = [preprocess_string(document) for document in documentslist] 

#remove low frequency terms (terms with fewer than 20) (brings it down to 35,244 from 953,010 tokens)
frequency = defaultdict(int)

for text in processed_corpus:
    for token in text:
        frequency[token] += 1

processed_corpus = [
    [token for token in text if frequency[token] > 10]
    for text in processed_corpus
]

#dictionary
dictionary = Dictionary(processed_corpus)

#filter extremes: get rid of words that appear in no fewer than 5 documents
dictionary.filter_extremes(no_below=5)
len(dictionary) #43,025

#Bag of Words
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]

In [16]:
#Time slice
counts = metadata[['Year', "paragraph_filetime"]].groupby(['Year']).agg(['count'])
timeslice=[i[0] for i in counts.values.tolist()]

In [17]:
#save
import pickle

f = open('/Users/carlyknight/Documents/Data/FuturePerfect/dynamic_topic_models/bow_corpus.pkl', 'wb') 
pickle.dump(bow_corpus, f , protocol=4)  

In [18]:
f = open('/Users/carlyknight/Documents/Data/FuturePerfect/dynamic_topic_models/timeslice.pkl', 'wb') 
pickle.dump(timeslice, f , protocol=4)  

In [19]:
f = open('/Users/carlyknight/Documents/Data/FuturePerfect/dynamic_topic_models/dictionary.pkl', 'wb') 
pickle.dump(dictionary, f , protocol=4)  

# DTM

In [20]:
model = DtmModel(dtm_path, bow_corpus, timeslice, num_topics=50,
                 id2word=dictionary, initialize_lda=True)

In [21]:
topics = model.show_topic(topicid=9, time=10, topn=10)
topics

[(0.04156752674081997, 'valu'),
 (0.0376338673406211, 'asset'),
 (0.0285629463923049, 'estim'),
 (0.023276627559869402, 'cost'),
 (0.021816637596394944, 'cash'),
 (0.01854876630028244, 'impair'),
 (0.017843035113375105, 'flow'),
 (0.01519685710753916, 'fair'),
 (0.012997546790459745, 'base'),
 (0.011756561266272806, 'loss')]

In [19]:
#https://towardsdatascience.com/topic-modeling-with-gensim-a5609cefccc

# save

In [22]:
f = open('/Users/carlyknight/Documents/Data/FuturePerfect/dynamic_topic_models/topic_para_futonly_50.pkl', 'wb') 
pickle.dump(model, f , protocol=4)          
f.close()                 